In [1]:
## Plan
# Take 2 of the same Point Cloud
# Shift one by xyz rpy
# Visualize together

# Try different combinations of args
# Recover the xyz rpy transformation

In [2]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def gen_volumetric_weld_templates_by_params():
    x_min, x_max = -4.0, 4.0
    y_min, y_max = -12.0, 12.0
    z_min, z_max = -1, 1
    resolution = 0.3
    stem_x_min, stem_x_max = 1.8, 3.2
    stem_y_min, stem_y_max = -8.0, 8.0
    top_center_y = 7.5
    bottom_center_y = -7.5
    radius = 3.20
    
    def dist(pt1, pt2):
        x1, y1 = pt1
        x2, y2 = pt2
        dx = x2-x1
        dy = y2-y1
        return np.sqrt(dx**2 + dy**2)

    points2 = []
    
    for z in np.arange(z_min, z_max, resolution):
        for x in np.arange(x_min, x_max, resolution):
            for y in np.arange(y_min, y_max, resolution):
                if dist((x,y), (0, top_center_y)) <= radius:
                    points2.append((x,y,z))
                elif dist((x,y), (0, bottom_center_y)) <= radius:
                    points2.append((x,y,z))
                elif stem_x_min<=x<=stem_x_max and stem_y_min<=y<=stem_y_max:
                    points2.append((x,y,z))

    points1 = [(-x, y, z) for x, y, z in points2]
    return {"[": points1, "]": points2}

## Inital PC coords
volumetric_templates = gen_volumetric_weld_templates_by_params()
pts1 = np.array(volumetric_templates["["])
# pts1

array([[ 3.1, -8.1, -1. ],
       [ 3.1, -7.8, -1. ],
       [ 3.1, -7.5, -1. ],
       ...,
       [-3.2,  7.2,  0.8],
       [-3.2,  7.5,  0.8],
       [-3.2,  7.8,  0.8]])

In [7]:
## Creating Secondary PC Coords by Translating init
T = np.identity(4)
T[:, -1] = np.array([10,-10, 20, 1])
pts2 = np.ones((pts1.shape[0], 4))
pts2[:, :-1] = pts1[:, :]
pts2 = (T@pts2.T).T
pts2 = pts2[:, :3]
# pts2

array([[ 13.1, -18.1,  19. ],
       [ 13.1, -17.8,  19. ],
       [ 13.1, -17.5,  19. ],
       ...,
       [  6.8,  -2.8,  20.8],
       [  6.8,  -2.5,  20.8],
       [  6.8,  -2.2,  20.8]])

In [14]:
print(f"pts1: {pts1.shape[0]}")
print(pts1)
print()

print("T")
print(T)
print()

print(f"pts2: {pts2.shape[0]}")
print(pts2)
print()

pts1: 6419
[[ 3.1 -8.1 -1. ]
 [ 3.1 -7.8 -1. ]
 [ 3.1 -7.5 -1. ]
 ...
 [-3.2  7.2  0.8]
 [-3.2  7.5  0.8]
 [-3.2  7.8  0.8]]

T
[[  1.   0.   0.  10.]
 [  0.   1.   0. -10.]
 [  0.   0.   1.  20.]
 [  0.   0.   0.   1.]]

pts2: 6419
[[ 13.1 -18.1  19. ]
 [ 13.1 -17.8  19. ]
 [ 13.1 -17.5  19. ]
 ...
 [  6.8  -2.8  20.8]
 [  6.8  -2.5  20.8]
 [  6.8  -2.2  20.8]]



In [10]:
## Combining the PCs into one
pts = np.vstack([pts1, pts2])

## Encapsulating as a PC object and visualizing
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pts)
o3d.visualization.draw_geometries([pcd])

In [60]:
# 2 separate PC objects
pcdTarget = o3d.geometry.PointCloud()
pcdTarget.points = o3d.utility.Vector3dVector(pts2)
pcdTarget.paint_uniform_color((.5, 0, 0))

pcdTemplate = o3d.geometry.PointCloud()
pcdTemplate.points = o3d.utility.Vector3dVector(pts1)
pcdTemplate.paint_uniform_color((0, 0, 0.5))

## duplicates
pcdTarget_dup = o3d.geometry.PointCloud()
pcdTarget_dup.points = o3d.utility.Vector3dVector(pts2)
pcdTarget_dup.paint_uniform_color((0, 0.5, 0))

pcdTemplate_dup = o3d.geometry.PointCloud()
pcdTemplate_dup.points = o3d.utility.Vector3dVector(pts1)
pcdTemplate_dup.paint_uniform_color((0, 0.5, 0))

## Visualizing together 
o3d.visualization.draw_geometries([pcdTarget, pcdTemplate, pcdTarget_dup, pcdTemplate_dup])
# o3d.visualization.draw_geometries([pcdTarget_dup, pcdTemplate_dup])

In [43]:
## Attempting Pt2Pt ICP

## Args: source, target, max_correspondence_distance, init_transform, 
##     estimation method, convergence criteria
init_transform = np.identity(4)
init_transform[:, -1] = np.array([9,-11, 20.5, 1])
dist_threshold = 0.7
result = o3d.pipelines.registration.registration_icp(pcdTemplate, pcdTarget, dist_threshold, init_transform,
                                                    o3d.pipelines.registration.TransformationEstimationPointToPoint())
result

RegistrationResult with fitness=8.912603e-01, inlier_rmse=2.895010e-01, and correspondence_set size of 5721
Access transformation to get result.

In [44]:
result.transformation

array([[ 9.99996807e-01, -1.34201495e-03,  2.14144869e-03,
         9.16744680e+00],
       [ 1.34167376e-03,  9.99999087e-01,  1.60755086e-04,
        -1.08763895e+01],
       [-2.14166247e-03, -1.57881448e-04,  9.99997694e-01,
         2.04774936e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [45]:
## Transform template by result Transform



In [67]:
def draw_registration_result(source, target, transformation1, transformation2):
    source_temp = o3d.geometry.PointCloud(source)
    source_temp2 = o3d.geometry.PointCloud(source)
    target_temp = o3d.geometry.PointCloud(target)
    
#     target_temp.transform(transformation)
#     target_temp.paint_uniform_color((0,0,0))
#     source_temp.transform(transformation)
    source_temp.transform(transformation1)
    source_temp.paint_uniform_color((0.4,0.4,0.4))
    source_temp2.transform(transformation2)
    source_temp2.paint_uniform_color((1,0.4,0.4))
#     target_temp2 = target_temp.transform(transformation)

    
#     o3d.visualization.draw_geometries([source_temp.to_legacy(), target_temp.to_legacy()])
#     o3d.visualization.draw_geometries([source_temp, target_temp, source, target])
#     o3d.visualization.draw_geometries([source_temp, source_temp2, source, target])
    o3d.visualization.draw_geometries([source_temp, source_temp2])
#     o3d.visualization.draw_geometries([source_temp, target_temp, source, target])
#     o3d.visualization.draw_geometries([source_temp, target_temp, target_temp2])

draw_registration_result(pcdTemplate, pcdTarget, init_transform, result.transformation)
# draw_registration_result(pcdTemplate, pcdTarget, result.transformation)
    

In [68]:
## need to fine tune the icp arguments for tighter alignment
## Scan multiple max correspondence thresholds and note the 
    ## The correspondeces, the rmse, the fitness and the time to process

In [ ]:
### Results Summary from different args used

